In [111]:
import pandas as pd
import os
import pyarrow.parquet as pq
from sklearn.feature_extraction.text import TfidfVectorizer
from rectools.dataset import Dataset
from rectools.models import PopularModel
from scipy.sparse import csr_matrix
from sentence_transformers import SentenceTransformer
import numpy as np
from datetime import datetime

In [243]:
parquet_file = pq.ParquetFile(r'train_dataset_cold_start_train\video_stat.parquet')
for i in parquet_file.iter_batches(batch_size=10000):
    df = i.to_pandas()
    break

In [244]:
parquet_file = pq.ParquetFile(r'train_dataset_cold_start_train\logs_df_2024-08-05.parquet')
for i in parquet_file.iter_batches(batch_size=1000000):
    interact = i.to_pandas()
    break

In [3]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [245]:
df.drop(columns='row_number', inplace=True)

In [246]:
df['txt'] = df['title'].apply(lambda x: 'Заголовок: ' + str(x) + '. ') + df['description'].apply(lambda x: 'Описание: ' + str(x) + '. ') + df['category_id'].apply(lambda x: 'Категория: ' + str(x) + '. ') 

In [7]:
emd_title = []

for i in range(0, len(df['txt'].tolist()), 10):
    emd_title.extend(list(model.encode(df['txt'].tolist()[i: i + 10])))
    print(i, i + 10)

emd = pd.DataFrame(emd_title)
emd.columns = [str(i) + '_embending' for i in emd.columns]

0 10
10 20
20 30
30 40
40 50
50 60
60 70
70 80
80 90
90 100
100 110
110 120
120 130
130 140
140 150
150 160
160 170
170 180
180 190
190 200
200 210
210 220
220 230
230 240
240 250
250 260
260 270
270 280
280 290
290 300
300 310
310 320
320 330
330 340
340 350
350 360
360 370
370 380
380 390
390 400
400 410
410 420
420 430
430 440
440 450
450 460
460 470
470 480
480 490
490 500
500 510
510 520
520 530
530 540
540 550
550 560
560 570
570 580
580 590
590 600
600 610
610 620
620 630
630 640
640 650
650 660
660 670
670 680
680 690
690 700
700 710
710 720
720 730
730 740
740 750
750 760
760 770
770 780
780 790
790 800
800 810
810 820
820 830
830 840
840 850
850 860
860 870
870 880
880 890
890 900
900 910
910 920
920 930
930 940
940 950
950 960
960 970
970 980
980 990
990 1000
1000 1010
1010 1020
1020 1030
1030 1040
1040 1050
1050 1060
1060 1070
1070 1080
1080 1090
1090 1100
1100 1110
1110 1120
1120 1130
1130 1140
1140 1150
1150 1160
1160 1170
1170 1180
1180 1190
1190 1200
1200 1210
1210 1220

In [247]:
sum(emd.isnull().sum())

0

In [248]:
df = df.reset_index(drop=True).join(emd)
df = df.rename(columns={'video_id': 'item_id'})

In [249]:
interact = interact.rename(columns={'video_id': 'item_id'})

In [250]:
interact

,event_timestamp,user_id,region,city,item_id,watchtime
0,2024-08-05 12:12:32+03:00,bd1b6b07-e6a2-46ad-aa58-990bb7e89252,288eb079-1c29-4749-ab7e-550451a4d765,da903ff0-9eb8-4e91-b25d-22c70f962ca2,0af133e1-4fbf-4e10-b9e8-19fbe1a011c3,32
1,2024-08-05 18:30:15+03:00,7db026fc-b55c-4580-bc26-e3df59ba6c7c,f58ab76f-2581-447a-b95b-5832e8757054,8449a572-fd10-44ae-a899-af656e350ad9,4f42aab2-6238-4381-bb53-36213916e912,340
2,2024-08-05 12:35:16+03:00,5c94a654-f376-4529-b463-b62b6b50e8ca,2c1f4ce4-66e0-4481-a0ed-244bf76fa83a,93a12888-fdf4-4889-a1ef-3ef0537d674d,8b106b84-f3f5-4c27-820c-7e1546b43614,12
3,2024-08-05 20:48:51+03:00,b972399a-c4b9-47fe-88fa-ccb1827cd131,f28a922a-68b9-46ce-8b52-c0be09413514,a269faea-d63c-4196-9963-00c62b5b3264,305389f3-1c8d-4026-9eef-e2b875a35380,2460
5,2024-08-05 05:19:19+03:00,807fb9e8-0aac-4dd2-836b-48797681f4ff,09714619-66dc-4faa-acdf-b2ae9dd2549d,51c8f786-5661-4a9c-8ab2-5d52f31095de,caeb71a4-f0f6-4c6a-ac79-31bdbb58fef1,87
...,...,...,...,...,...,...
1072560,2024-08-05 18:55:28+03:00,b995f499-47d5-4b2e-9179-ddf3f8d83ae4,01c33221-6c23-44be-a52e-228a8dc720e6,08127749-3fac-4ec6-9fc6-8a9df93eefca,40345577-19fc-4342-8f4f-d98c6f0c446a,20
1072561,2024-08-05 14:55:09+03:00,b7a152f5-0abe-4cf5-bfc9-0e04eb576d19,495bd532-05ff-4d6b-90ae-0f3c8a0b3ec5,ae5bcc6c-0a3a-4ee0-abbf-11139c6b3a90,06753e5e-0c15-4e60-bc9c-ee64fc743c99,0
1072562,2024-08-05 20:36:58+03:00,f4ef8007-db76-4e36-b601-54ba7bba1660,f9afd777-12f0-4a46-ac90-9549d8710b2d,4cbf1ddf-0575-44a7-9d07-2f1400c498a6,7b6f4d2f-4ebd-4e2d-bc17-55f0fc0b3aa1,1
1072563,2024-08-05 20:56:12+03:00,817f6fd4-95b1-4b49-9068-9fe44edf0b80,f9afd777-12f0-4a46-ac90-9549d8710b2d,4cbf1ddf-0575-44a7-9d07-2f1400c498a6,e3cd387e-9733-40da-baab-41ce805e7e80,0


In [251]:
interactions = interact.merge(df[['v_duration', 'item_id']])
interactions['weight'] = interactions['watchtime'] / interactions['v_duration'] * 5
interactions = interactions.replace(np.inf, np.nan)
interactions.dropna(inplace=True)
interactions['weight'] = interactions['weight'].apply(lambda x: 5 if x > 5 else 0 if x < 0 else x)
interactions['weight'] = interactions['weight'].apply(round)
interactions = interactions.drop(columns=['v_duration'])

In [252]:
interactions

,event_timestamp,user_id,region,city,item_id,watchtime,weight
0,2024-08-05 12:35:16+03:00,5c94a654-f376-4529-b463-b62b6b50e8ca,2c1f4ce4-66e0-4481-a0ed-244bf76fa83a,93a12888-fdf4-4889-a1ef-3ef0537d674d,8b106b84-f3f5-4c27-820c-7e1546b43614,12,0
1,2024-08-05 19:32:59+03:00,79c47b82-800e-47f0-ac54-8b3abdbac825,2c7ecf72-4569-4066-b75e-0465aac19cee,98be10a3-2dd5-4336-9d1f-dc7812bd6e9a,a58305b8-826e-4dfd-9d77-578a116e0f7d,0,0
2,2024-08-05 18:47:13+03:00,eeec28ac-c27c-4606-86e0-789f8c4efc78,f9afd777-12f0-4a46-ac90-9549d8710b2d,4cbf1ddf-0575-44a7-9d07-2f1400c498a6,683a2978-5e84-41c1-9cb2-2cd853195700,31,5
3,2024-08-05 15:55:05+03:00,be51b049-dee8-4993-ad80-f78cc8ba21dc,21c82c6d-431a-49cc-a628-7cea037c7602,f9188a3c-0c71-4a7b-8940-b943237b9820,86fa8122-0b3d-4c0d-a7ee-6d816c931dad,192,5
4,2024-08-05 02:05:47+03:00,71400216-e537-4a42-a421-d6da58645a95,d2a0aecc-d3ee-442b-a3f6-6fbdf2ec4187,81f25e23-4db0-4823-a5f6-41b5999023c1,91047ab4-56df-4bff-902d-90152d07a1d7,661,5
...,...,...,...,...,...,...,...
11165,2024-08-05 13:31:37+03:00,0fc8b807-c7fb-4ea1-bd3a-485a2d084520,1f6e5a30-8e20-465f-a110-ac2a9ea5f12c,99016c33-ffab-4af0-a971-c4dfd9bada65,a91e4c8e-4466-410d-89be-2f3438da413b,0,0
11166,2024-08-05 19:43:58+03:00,cbbd197e-9d07-4874-b9b9-c6753a505469,09714619-66dc-4faa-acdf-b2ae9dd2549d,51c8f786-5661-4a9c-8ab2-5d52f31095de,6a3b2d6b-86c3-4234-bf63-4cf704980abb,0,0
11167,2024-08-05 23:17:06+03:00,519ab37d-c05d-442e-a849-6cc3e808f3b1,95f75a27-4c81-434f-989b-4945b5040242,2ce79733-f9a1-4116-b09a-53b16385db38,94e52d25-7aed-441a-bc8c-db7f4b75677a,2326,2
11168,2024-08-05 12:32:31+03:00,f1d32e74-20a5-4e2c-aff2-db0bf2add514,f9afd777-12f0-4a46-ac90-9549d8710b2d,4cbf1ddf-0575-44a7-9d07-2f1400c498a6,79c98fbb-8ce9-4a08-b6bb-4da57c91715b,1262,5


In [253]:
interactions['datetime'] = datetime(year=2024, month=8, day=5)

In [254]:
interactions = interactions.drop(columns=['watchtime', 'region', 'city'])

In [255]:
interactions

,event_timestamp,user_id,item_id,weight,datetime
0,2024-08-05 12:35:16+03:00,5c94a654-f376-4529-b463-b62b6b50e8ca,8b106b84-f3f5-4c27-820c-7e1546b43614,0,2024-08-05
1,2024-08-05 19:32:59+03:00,79c47b82-800e-47f0-ac54-8b3abdbac825,a58305b8-826e-4dfd-9d77-578a116e0f7d,0,2024-08-05
2,2024-08-05 18:47:13+03:00,eeec28ac-c27c-4606-86e0-789f8c4efc78,683a2978-5e84-41c1-9cb2-2cd853195700,5,2024-08-05
3,2024-08-05 15:55:05+03:00,be51b049-dee8-4993-ad80-f78cc8ba21dc,86fa8122-0b3d-4c0d-a7ee-6d816c931dad,5,2024-08-05
4,2024-08-05 02:05:47+03:00,71400216-e537-4a42-a421-d6da58645a95,91047ab4-56df-4bff-902d-90152d07a1d7,5,2024-08-05
...,...,...,...,...,...
11165,2024-08-05 13:31:37+03:00,0fc8b807-c7fb-4ea1-bd3a-485a2d084520,a91e4c8e-4466-410d-89be-2f3438da413b,0,2024-08-05
11166,2024-08-05 19:43:58+03:00,cbbd197e-9d07-4874-b9b9-c6753a505469,6a3b2d6b-86c3-4234-bf63-4cf704980abb,0,2024-08-05
11167,2024-08-05 23:17:06+03:00,519ab37d-c05d-442e-a849-6cc3e808f3b1,94e52d25-7aed-441a-bc8c-db7f4b75677a,2,2024-08-05
11168,2024-08-05 12:32:31+03:00,f1d32e74-20a5-4e2c-aff2-db0bf2add514,79c98fbb-8ce9-4a08-b6bb-4da57c91715b,5,2024-08-05


In [256]:
df = df.drop(columns=['title', 'description', 'txt', 'category_id', 'author_id', 'v_pub_datetime'])

In [257]:
cols = df.columns.tolist()
cols.remove('item_id')

In [258]:
# Преобразуем video_stat в "длинный" формат (feature-value)
item_features_df = df.melt(id_vars=['item_id'], value_vars=cols, 
                                   var_name='feature', value_name='value')

In [259]:
item_features_df

,item_id,feature,value
0,973f24c0-15b2-434f-8740-4e2726f79c30,v_total_comments,0.000000
1,1a3f527c-44b2-4f7e-9641-26f078edbf2e,v_total_comments,0.000000
2,2f563d09-45ef-4e27-8da0-daa74235a2e7,v_total_comments,0.000000
3,154fac0e-1794-4406-afa0-61f688e3a764,v_total_comments,0.000000
4,470b4e3e-e06d-4370-80dc-34d6a78b22db,v_total_comments,0.000000
...,...,...,...
4199995,4c4322fa-7b1d-493c-975d-70d1f253eb8d,383_embending,0.036212
4199996,d54105bc-3d42-4dcc-b2c1-ed0c66c0c142,383_embending,-0.018523
4199997,3622e657-3241-48e8-9f1e-be1b8b7e3c9c,383_embending,-0.021324
4199998,d3cea5a8-20a8-4460-88d0-714e3ffad7df,383_embending,0.030851


In [260]:
dataset = Dataset.construct(
    interactions_df=interactions,
    item_features_df=item_features_df
)

In [261]:
model = PopularModel()
model.fit(dataset)

In [287]:
for id in interactions['user_id'].unique():
    recommendations = model.recommend(
    users=[id],
    dataset=dataset,
    k=10,
    filter_viewed=True
    )
    print(recommendations['item_id'].tolist())

['8380b4f2-4c45-45f4-96ec-cc8487e7954e', '989976b7-3afd-48af-9797-970b0c9e53a5', '6977fc38-00ce-4571-8290-17698ba2d490', 'a09d996f-8527-474a-bec9-cae2e11fd9da', '683a2978-5e84-41c1-9cb2-2cd853195700', '46fbd9a1-8914-475a-a7a5-f4557b92910d', '0c3ce65a-d7cf-4e1d-8e8b-b81f67ee24fc', '80d831cd-ac9e-4cef-a086-dc92abf8c621', '0774fd74-01a1-4a88-91c5-65d113fb0349', '95f8ab2a-4a0c-4b51-85f5-1cfe5a5fef1d']
['8380b4f2-4c45-45f4-96ec-cc8487e7954e', '989976b7-3afd-48af-9797-970b0c9e53a5', '6977fc38-00ce-4571-8290-17698ba2d490', 'a09d996f-8527-474a-bec9-cae2e11fd9da', '683a2978-5e84-41c1-9cb2-2cd853195700', '46fbd9a1-8914-475a-a7a5-f4557b92910d', '0c3ce65a-d7cf-4e1d-8e8b-b81f67ee24fc', '80d831cd-ac9e-4cef-a086-dc92abf8c621', '0774fd74-01a1-4a88-91c5-65d113fb0349', '95f8ab2a-4a0c-4b51-85f5-1cfe5a5fef1d']
['8380b4f2-4c45-45f4-96ec-cc8487e7954e', '989976b7-3afd-48af-9797-970b0c9e53a5', '6977fc38-00ce-4571-8290-17698ba2d490', 'a09d996f-8527-474a-bec9-cae2e11fd9da', '46fbd9a1-8914-475a-a7a5-f4557b92910d

KeyboardInterrupt: 